In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('seaborn-white')
# colorblind safe
plt.style.use('seaborn-colorblind')
plt.style.use('tableau-colorblind10')

# width and precision for f strings
width = 10
precision = 4

# default sizes for plots
# https://matplotlib.org/3.3.0/tutorials/introductory/customizing.html#customizing-with-matplotlibrc-files
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 16
plt.rcParams['legend.fontsize'] = 'large'
plt.rcParams['figure.titlesize'] = 'medium'
plt.rcParams['lines.linewidth'] = 2

# other settings
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option("display.precision", 3)
np.set_printoptions(precision=3, suppress=True)
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', None)
%config IPCompleter.greedy=True


In [2]:
# get data
USdata = pd.read_csv('data/iOverlander Places -  2020-08-22.csv')

In [19]:
USdata.loc[1699:1701,:]

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center
1699,NaN,Coachella Vista Point,Wild Camping,A little chunk of BLM land sitting just above ...,33.6482,-116.4004,727.0000,2020-04-14 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1700,"Cam Playa, Aguada, 00602, Puerto Rico",Pico de Piedra Beach,Informal Campsite,"Nice and Beautiful west coast beach , Great Re...",18.3842,-67.2122,nan,2017-12-22 00:00:00 UTC,Yes,Yes,Yes - Average,Yes,nan,Yes,Warm,Potable,Running Water,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1701,NaN,Under the Bridge,Informal Campsite,Several parking spaces (for cars) under/next t...,30.2885,-81.4195,0.1891,2020-05-16 00:00:00 UTC,Yes,No,No,No,nan,No,Warm,No,Running Water,No,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
USdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   1413 non-null   object 
 1   Name                       8790 non-null   object 
 2   Category                   8790 non-null   object 
 3   Description                8790 non-null   object 
 4   Latitude                   8790 non-null   float64
 5   Longitude                  8790 non-null   float64
 6   Altitude                   8567 non-null   float64
 7   Date verified              8790 non-null   object 
 8   Open                       8790 non-null   object 
 9   Electricity                7987 non-null   object 
 10  Wifi                       7987 non-null   object 
 11  Kitchen                    7987 non-null   object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 7987 non-null   objec

In [3]:
import os
G_MAPS_API = os.environ.get("GOOGLE_STATICMAPS_API")

In [7]:
import random, time

In [4]:
import googlemaps

In [5]:
gmaps = googlemaps.Client(G_MAPS_API)

In [6]:
n_requests = 0

In [31]:
def get_state_zip(df):
    '''
    gets zip code and state and inserts into df
    '''
    zips = []
    states = []
    
    global n_requests
    
    for i, site in df.iterrows():
        result = None
        print(f'Pulling request {i}, total API requests so far = {n_requests}')
        
        # get latlong in right format
        # temp save coords
        lat = str(site['Latitude'])
        long = str(site['Longitude'])
        latlong = lat + ',' + long
        #rint(latlong)
        # get geocode data
        res_type = 'postal_code' # administrative_area_level_1|
        result = gmaps.reverse_geocode(latlong, result_type=res_type)
        # increase counter
        n_requests += 1
        #rint(result)
        if result:
            # pull out things we need
            zip_code = result[0]['address_components'][0]['short_name']
            # sometimes the index isn't the same
            #state = result[0]['address_components'][3]['short_name']
            types = ['administrative_area_level_1', 'political']
            statethingy = [d.items() for d in result[0]['address_components'] if d['types'] == types]
            # sometimes there isn't this type (like Puerto Rico)
            #print(statethingy)
            if statethingy:
                state = [x[1] for x in statethingy[0] if x[0] == 'short_name'][0]
    #             for item in statethingy[0]:
    #                 if item[0] == 'short_name':
    #                     state = item[1]
            else:
                # if it didn't work for this row
                zip_code = ''
                state = ''
        else:
            # if it didn't work for this row
            zip_code = ''
            state = ''
        
        # append
        zips.append(zip_code)
        states.append(state)
        
        # wait a bit before next request
        wait_time = random.randint(1, 3)
        print(f'waiting for: {wait_time} seconds')
        time.sleep(wait_time) # in seconds
    
    return zips, states

In [14]:
# if we have a location we don't need to get it
USdata_need_loc = USdata[pd.isna(USdata['Location'])].copy()

In [15]:
len(USdata_need_loc)

7377

In [ ]:
# if we have location, we can assign states like this:
# USdata_CO = USdata_CO[USdata_CO['Location'].str.contains(' CO', case=True, regex=False)]

In [34]:
# do first 500 for now
USdata_500 = USdata_need_loc.copy().head(500)

In [35]:
USdata_500.head()

,index,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center
0,1221,NaN,Springer Lake,Wild Camping,Dispersed camping on shore of small lake. Ceme...,36.4088,-104.6526,"1,788.0000",2020-04-01 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,Pit Toilets,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1,1222,NaN,"large flat spot - on route 9 , before the 89 j...",Wild Camping,Be careful low branches. Not suitable for rvs...,37.2615,-112.7791,"1,811.0000",2019-05-29 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2,1223,NaN,Zigzag falls,Wild Camping,Lots of small camping spots along Kiwanis Camp...,45.3141,-121.8082,944.3531,2019-06-02 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3,1224,NaN,Topsy Grade Rd,Wild Camping,Open woodlands along the reservoir with plenty...,42.1318,-122.0332,"1,194.4113",2019-09-27 00:00:00 UTC,Yes,No,No,No,nan,No,No,Non-Potable,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4,1225,NaN,Lava tubes,Wild Camping,Pit toilet picnic table site,34.9446,-108.1069,"2,324.8057",2017-09-24 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,Pit Toilets,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# test with problem one
problems = USdata.loc[1698:1702,:]

In [32]:
zips, states = get_state_zip(problems)

Pulling request 1698, total API requests so far = 1722
waiting for: 2 seconds
Pulling request 1699, total API requests so far = 1723
waiting for: 2 seconds
Pulling request 1700, total API requests so far = 1724
waiting for: 2 seconds
Pulling request 1701, total API requests so far = 1725
waiting for: 1 seconds


KeyboardInterrupt: 

In [28]:
zips

['98901', '92561', '', '32250', '90755']

In [36]:
zips, states = get_state_zip(USdata_500)

Pulling request 0, total API requests so far = 1739
waiting for: 2 seconds
Pulling request 1, total API requests so far = 1740
waiting for: 1 seconds
Pulling request 2, total API requests so far = 1741
waiting for: 3 seconds
Pulling request 3, total API requests so far = 1742
waiting for: 2 seconds
Pulling request 4, total API requests so far = 1743
waiting for: 2 seconds
Pulling request 5, total API requests so far = 1744
waiting for: 3 seconds
Pulling request 6, total API requests so far = 1745
waiting for: 2 seconds
Pulling request 7, total API requests so far = 1746
waiting for: 2 seconds
Pulling request 8, total API requests so far = 1747
waiting for: 2 seconds
Pulling request 9, total API requests so far = 1748
waiting for: 1 seconds
Pulling request 10, total API requests so far = 1749
waiting for: 2 seconds
Pulling request 11, total API requests so far = 1750
waiting for: 3 seconds
Pulling request 12, total API requests so far = 1751
waiting for: 3 seconds
Pulling request 13, to

Pulling request 108, total API requests so far = 1847
waiting for: 3 seconds
Pulling request 109, total API requests so far = 1848
waiting for: 1 seconds
Pulling request 110, total API requests so far = 1849
waiting for: 2 seconds
Pulling request 111, total API requests so far = 1850
waiting for: 2 seconds
Pulling request 112, total API requests so far = 1851
waiting for: 1 seconds
Pulling request 113, total API requests so far = 1852
waiting for: 3 seconds
Pulling request 114, total API requests so far = 1853
waiting for: 2 seconds
Pulling request 115, total API requests so far = 1854
waiting for: 2 seconds
Pulling request 116, total API requests so far = 1855
waiting for: 2 seconds
Pulling request 117, total API requests so far = 1856
waiting for: 3 seconds
Pulling request 118, total API requests so far = 1857
waiting for: 2 seconds
Pulling request 119, total API requests so far = 1858
waiting for: 3 seconds
Pulling request 120, total API requests so far = 1859
waiting for: 2 seconds

waiting for: 1 seconds
Pulling request 215, total API requests so far = 1954
waiting for: 3 seconds
Pulling request 216, total API requests so far = 1955
waiting for: 2 seconds
Pulling request 217, total API requests so far = 1956
waiting for: 1 seconds
Pulling request 218, total API requests so far = 1957
waiting for: 1 seconds
Pulling request 219, total API requests so far = 1958
waiting for: 2 seconds
Pulling request 220, total API requests so far = 1959
waiting for: 1 seconds
Pulling request 221, total API requests so far = 1960
waiting for: 3 seconds
Pulling request 222, total API requests so far = 1961
waiting for: 3 seconds
Pulling request 223, total API requests so far = 1962
waiting for: 2 seconds
Pulling request 224, total API requests so far = 1963
waiting for: 1 seconds
Pulling request 225, total API requests so far = 1964
waiting for: 3 seconds
Pulling request 226, total API requests so far = 1965
waiting for: 2 seconds
Pulling request 227, total API requests so far = 1966

Pulling request 322, total API requests so far = 2061
waiting for: 2 seconds
Pulling request 323, total API requests so far = 2062
waiting for: 1 seconds
Pulling request 324, total API requests so far = 2063
waiting for: 3 seconds
Pulling request 325, total API requests so far = 2064
waiting for: 2 seconds
Pulling request 326, total API requests so far = 2065
waiting for: 3 seconds
Pulling request 327, total API requests so far = 2066
waiting for: 3 seconds
Pulling request 328, total API requests so far = 2067
waiting for: 2 seconds
Pulling request 329, total API requests so far = 2068
waiting for: 3 seconds
Pulling request 330, total API requests so far = 2069
waiting for: 2 seconds
Pulling request 331, total API requests so far = 2070
waiting for: 1 seconds
Pulling request 332, total API requests so far = 2071
waiting for: 3 seconds
Pulling request 333, total API requests so far = 2072
waiting for: 3 seconds
Pulling request 334, total API requests so far = 2073
waiting for: 3 seconds

Pulling request 429, total API requests so far = 2168
waiting for: 1 seconds
Pulling request 430, total API requests so far = 2169
waiting for: 1 seconds
Pulling request 431, total API requests so far = 2170
waiting for: 1 seconds
Pulling request 432, total API requests so far = 2171
waiting for: 2 seconds
Pulling request 433, total API requests so far = 2172
waiting for: 2 seconds
Pulling request 434, total API requests so far = 2173
waiting for: 1 seconds
Pulling request 435, total API requests so far = 2174
waiting for: 1 seconds
Pulling request 436, total API requests so far = 2175
waiting for: 3 seconds
Pulling request 437, total API requests so far = 2176
waiting for: 1 seconds
Pulling request 438, total API requests so far = 2177
waiting for: 3 seconds
Pulling request 439, total API requests so far = 2178
waiting for: 2 seconds
Pulling request 440, total API requests so far = 2179
waiting for: 1 seconds
Pulling request 441, total API requests so far = 2180
waiting for: 2 seconds

In [37]:
len(zips)

500

In [38]:
len(states)

500

In [39]:
# save
zips_pd = pd.Series(zips, name='zip_codes') 
states_pd = pd.Series(states, name='states') 
zips_pd.to_csv('data/zip_codes.csv')
states_pd.to_csv('data/states.csv')

In [40]:
USdata_zip_states = USdata_500.copy()

In [41]:
USdata_zip_states['zip_code'] = zips
USdata_zip_states['State'] = states

In [42]:
USdata_zip_states.head()

,index,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center,zip_code,State
0,1221,NaN,Springer Lake,Wild Camping,Dispersed camping on shore of small lake. Ceme...,36.4088,-104.6526,"1,788.0000",2020-04-01 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,Pit Toilets,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,87747,NM
1,1222,NaN,"large flat spot - on route 9 , before the 89 j...",Wild Camping,Be careful low branches. Not suitable for rvs...,37.2615,-112.7791,"1,811.0000",2019-05-29 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,84755,UT
2,1223,NaN,Zigzag falls,Wild Camping,Lots of small camping spots along Kiwanis Camp...,45.3141,-121.8082,944.3531,2019-06-02 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97041,OR
3,1224,NaN,Topsy Grade Rd,Wild Camping,Open woodlands along the reservoir with plenty...,42.1318,-122.0332,"1,194.4113",2019-09-27 00:00:00 UTC,Yes,No,No,No,nan,No,No,Non-Potable,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97627,OR
4,1225,NaN,Lava tubes,Wild Camping,Pit toilet picnic table site,34.9446,-108.1069,"2,324.8057",2017-09-24 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,Pit Toilets,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,87020,NM


In [43]:
USdata_CO = USdata_zip_states[USdata_zip_states['State'] == 'CO'].copy()

In [44]:
USdata_CO.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 6 to 488
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      16 non-null     int64  
 1   Location                   0 non-null      object 
 2   Name                       16 non-null     object 
 3   Category                   16 non-null     object 
 4   Description                16 non-null     object 
 5   Latitude                   16 non-null     float64
 6   Longitude                  16 non-null     float64
 7   Altitude                   16 non-null     float64
 8   Date verified              16 non-null     object 
 9   Open                       16 non-null     object 
 10  Electricity                15 non-null     object 
 11  Wifi                       15 non-null     object 
 12  Kitchen                    15 non-null     object 
 13  Parking                    0 non-null      float64


In [45]:
USdata_500.tail()

,index,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center
495,1862,NaN,"Veterans Memorial Park and area, Lake Charles",Informal Campsite,A beautiful park on the Lake Charles waterfron...,30.2313,-93.2212,44.0000,2020-02-25 00:00:00 UTC,Yes,Yes - Not at Sites,Yes - Slow,No,nan,No,No,Yes,Running Water,No,No,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
496,1863,NaN,Street near beach,Wild Camping,Just a street where u can park for free as lon...,34.0290,-118.5142,36.8156,2018-10-03 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
497,1866,NaN,Off of hole in the rock road,Wild Camping,"Next to a gated cattle run. Somewhat flat, but...",37.6367,-111.4475,"1,613.5075",2020-05-03 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
498,1867,NaN,Free campsite in Coconico NF,Wild Camping,Free campsite. No amenities. Very quiet.,35.4160,-111.4000,"1,707.8994",2018-04-11 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
499,1868,NaN,Chevron gas station with very large paved over...,Informal Campsite,Football size paved parking used by large RV a...,35.4607,-114.9171,"1,084.3755",2018-04-13 00:00:00 UTC,Yes,No,No,No,nan,Yes,Unknown,No,Running Water,Yes,No,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
USdata_501_1000 = USdata_need_loc.iloc[500:1000,:]

In [51]:
len(USdata_501_1000)

500

In [52]:
zips, states = get_state_zip(USdata_501_1000)

Pulling request 1869, total API requests so far = 2239
waiting for: 3 seconds
Pulling request 1870, total API requests so far = 2240
waiting for: 1 seconds
Pulling request 1871, total API requests so far = 2241
waiting for: 3 seconds
Pulling request 1872, total API requests so far = 2242
waiting for: 1 seconds
Pulling request 1873, total API requests so far = 2243
waiting for: 3 seconds
Pulling request 1875, total API requests so far = 2244
waiting for: 3 seconds
Pulling request 1876, total API requests so far = 2245
waiting for: 2 seconds
Pulling request 1877, total API requests so far = 2246
waiting for: 3 seconds
Pulling request 1878, total API requests so far = 2247
waiting for: 2 seconds
Pulling request 1879, total API requests so far = 2248
waiting for: 2 seconds
Pulling request 1880, total API requests so far = 2249
waiting for: 3 seconds
Pulling request 1881, total API requests so far = 2250
waiting for: 2 seconds
Pulling request 1882, total API requests so far = 2251
waiting f

Pulling request 2007, total API requests so far = 2345
waiting for: 1 seconds
Pulling request 2008, total API requests so far = 2346
waiting for: 2 seconds
Pulling request 2009, total API requests so far = 2347
waiting for: 3 seconds
Pulling request 2010, total API requests so far = 2348
waiting for: 3 seconds
Pulling request 2011, total API requests so far = 2349
waiting for: 2 seconds
Pulling request 2012, total API requests so far = 2350
waiting for: 3 seconds
Pulling request 2013, total API requests so far = 2351
waiting for: 3 seconds
Pulling request 2014, total API requests so far = 2352
waiting for: 1 seconds
Pulling request 2015, total API requests so far = 2353
waiting for: 3 seconds
Pulling request 2016, total API requests so far = 2354
waiting for: 1 seconds
Pulling request 2017, total API requests so far = 2355
waiting for: 3 seconds
Pulling request 2018, total API requests so far = 2356
waiting for: 3 seconds
Pulling request 2019, total API requests so far = 2357
waiting f

Pulling request 2121, total API requests so far = 2451
waiting for: 2 seconds
Pulling request 2122, total API requests so far = 2452
waiting for: 3 seconds
Pulling request 2123, total API requests so far = 2453
waiting for: 1 seconds
Pulling request 2124, total API requests so far = 2454
waiting for: 2 seconds
Pulling request 2125, total API requests so far = 2455
waiting for: 1 seconds
Pulling request 2126, total API requests so far = 2456
waiting for: 1 seconds
Pulling request 2127, total API requests so far = 2457
waiting for: 3 seconds
Pulling request 2128, total API requests so far = 2458
waiting for: 3 seconds
Pulling request 2129, total API requests so far = 2459
waiting for: 3 seconds
Pulling request 2130, total API requests so far = 2460
waiting for: 2 seconds
Pulling request 2131, total API requests so far = 2461
waiting for: 3 seconds
Pulling request 2132, total API requests so far = 2462
waiting for: 3 seconds
Pulling request 2133, total API requests so far = 2463
waiting f

Pulling request 2229, total API requests so far = 2557
waiting for: 1 seconds
Pulling request 2230, total API requests so far = 2558
waiting for: 3 seconds
Pulling request 2231, total API requests so far = 2559
waiting for: 2 seconds
Pulling request 2232, total API requests so far = 2560
waiting for: 1 seconds
Pulling request 2233, total API requests so far = 2561
waiting for: 1 seconds
Pulling request 2234, total API requests so far = 2562
waiting for: 3 seconds
Pulling request 2235, total API requests so far = 2563
waiting for: 2 seconds
Pulling request 2236, total API requests so far = 2564
waiting for: 2 seconds
Pulling request 2237, total API requests so far = 2565
waiting for: 3 seconds
Pulling request 2238, total API requests so far = 2566
waiting for: 3 seconds
Pulling request 2239, total API requests so far = 2567
waiting for: 3 seconds
Pulling request 2240, total API requests so far = 2568
waiting for: 2 seconds
Pulling request 2241, total API requests so far = 2569
waiting f

Pulling request 2335, total API requests so far = 2663
waiting for: 3 seconds
Pulling request 2336, total API requests so far = 2664
waiting for: 3 seconds
Pulling request 2337, total API requests so far = 2665
waiting for: 2 seconds
Pulling request 2338, total API requests so far = 2666
waiting for: 2 seconds
Pulling request 2339, total API requests so far = 2667
waiting for: 2 seconds
Pulling request 2340, total API requests so far = 2668
waiting for: 1 seconds
Pulling request 2341, total API requests so far = 2669
waiting for: 1 seconds
Pulling request 2342, total API requests so far = 2670
waiting for: 2 seconds
Pulling request 2343, total API requests so far = 2671
waiting for: 3 seconds
Pulling request 2344, total API requests so far = 2672
waiting for: 2 seconds
Pulling request 2345, total API requests so far = 2673
waiting for: 3 seconds
Pulling request 2346, total API requests so far = 2674
waiting for: 2 seconds
Pulling request 2347, total API requests so far = 2675
waiting f

In [53]:
zips

['31401',
 '70460',
 '26296',
 '26296',
 '26264',
 '',
 '89406',
 '95043',
 '87322',
 '89301',
 '89510',
 '87499',
 '84535',
 '97907',
 '04042',
 '08865',
 '92358',
 '85741',
 '82930',
 '86335',
 '86327',
 '90265',
 '93240',
 '37201',
 '28721',
 '22015',
 '27410',
 '85326',
 '96056',
 '03584',
 '87401',
 '22664',
 '87821',
 '86336',
 '80504',
 '70124',
 '22192',
 '78418',
 '13209',
 '85192',
 '89040',
 '',
 '84745',
 '99350',
 '93265',
 '86023',
 '',
 '93243',
 '93274',
 '95380',
 '14569',
 '15219',
 '83213',
 '',
 '74834',
 '81146',
 '81122',
 '93449',
 '94706',
 '77984',
 '97049',
 '98331',
 '99348',
 '88061',
 '08550',
 '84741',
 '17013',
 '89039',
 '97041',
 '60616',
 '58563',
 '87823',
 '20560',
 '85326',
 '17101',
 '63366',
 '92364',
 '75670',
 '82838',
 '81321',
 '28531',
 '90069',
 '82711',
 '42029',
 '62656',
 '32566',
 '84531',
 '07069',
 '81505',
 '30545',
 '36804',
 '08332',
 '89406',
 '98503',
 '29069',
 '',
 '08008',
 '89310',
 '74562',
 '49014',
 'N2C 1W9',
 '28771',
 '9

In [54]:
# save
zips_pd_next500 = pd.Series(zips, name='zip_codes') 
states_pd_next500 = pd.Series(states, name='states') 
zips_pd_next500.to_csv('data/zip_codes_next500.csv')
states_pd_next500.to_csv('data/states_next500.csv')

In [57]:
zips_pd_next500

0      31401
1      70460
2      26296
3      26296
4      26264
       ...  
495    59486
496         
497    89408
498    99603
499    97465
Name: zip_codes, Length: 500, dtype: object

In [58]:
USdata_501_1000['zip_code'] = zips
USdata_501_1000['State'] = states

/usr/local/anaconda3/envs/py377_maps/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/anaconda3/envs/py377_maps/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [59]:
USdata_501_1000.head()

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center,zip_code,State
1869,NaN,Savannah International Trade and Convention Ce...,Informal Campsite,Quiet place right next to the parking lot of t...,32.0859,-81.0883,3.5926,2020-03-18 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,31401,GA
1870,NaN,Big branch refuge,Informal Campsite,Gravy parking lot before a national wildlife r...,30.2721,-89.8706,-7.6922,2018-04-13 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,70460,LA
1871,NaN,Monongahela NF - Gandy Creek,Wild Camping,Dispersed camping on Gandy Creek. Alongside r...,38.7809,-79.5618,918.8701,2019-07-17 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,26296,WV
1872,NaN,Monongahela NF,Wild Camping,Grant’s Branch. Absolutely beautiful and peac...,38.7278,-79.6143,"1,034.2505",2018-04-13 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,26296,WV
1873,NaN,Durbin - Greenbrier River,Wild Camping,Quiet spot off the road amongst the rhododendr...,38.5273,-79.8409,825.5439,2018-08-02 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,26264,WV


In [60]:
USdata_1001 = USdata_need_loc.iloc[1000:,:]

In [ ]:
zips, states = get_state_zip(USdata_1001)

Pulling request 2411, total API requests so far = 2739
waiting for: 3 seconds
Pulling request 2412, total API requests so far = 2740
waiting for: 1 seconds
Pulling request 2413, total API requests so far = 2741
waiting for: 3 seconds
Pulling request 2414, total API requests so far = 2742
waiting for: 2 seconds
Pulling request 2415, total API requests so far = 2743
waiting for: 2 seconds
Pulling request 2416, total API requests so far = 2744
waiting for: 3 seconds
Pulling request 2417, total API requests so far = 2745
waiting for: 2 seconds
Pulling request 2418, total API requests so far = 2746
waiting for: 1 seconds
Pulling request 2419, total API requests so far = 2747
waiting for: 1 seconds
Pulling request 2420, total API requests so far = 2748
waiting for: 1 seconds
Pulling request 2421, total API requests so far = 2749
waiting for: 1 seconds
Pulling request 2422, total API requests so far = 2750
waiting for: 2 seconds
Pulling request 2423, total API requests so far = 2751
waiting f

Pulling request 2517, total API requests so far = 2845
waiting for: 3 seconds
Pulling request 2518, total API requests so far = 2846
waiting for: 2 seconds
Pulling request 2519, total API requests so far = 2847
waiting for: 1 seconds
Pulling request 2520, total API requests so far = 2848
waiting for: 1 seconds
Pulling request 2521, total API requests so far = 2849
waiting for: 1 seconds
Pulling request 2522, total API requests so far = 2850
waiting for: 1 seconds
Pulling request 2523, total API requests so far = 2851
waiting for: 3 seconds
Pulling request 2524, total API requests so far = 2852
waiting for: 2 seconds
Pulling request 2525, total API requests so far = 2853
waiting for: 1 seconds
Pulling request 2526, total API requests so far = 2854
waiting for: 1 seconds
Pulling request 2527, total API requests so far = 2855
waiting for: 2 seconds
Pulling request 2528, total API requests so far = 2856
waiting for: 3 seconds
Pulling request 2529, total API requests so far = 2857
waiting f

waiting for: 1 seconds
Pulling request 2623, total API requests so far = 2951
waiting for: 3 seconds
Pulling request 2624, total API requests so far = 2952
waiting for: 1 seconds
Pulling request 2625, total API requests so far = 2953
waiting for: 3 seconds
Pulling request 2626, total API requests so far = 2954
waiting for: 3 seconds
Pulling request 2627, total API requests so far = 2955
waiting for: 3 seconds
Pulling request 2628, total API requests so far = 2956
waiting for: 3 seconds
Pulling request 2629, total API requests so far = 2957
waiting for: 2 seconds
Pulling request 2630, total API requests so far = 2958
waiting for: 3 seconds
Pulling request 2631, total API requests so far = 2959
waiting for: 2 seconds
Pulling request 2632, total API requests so far = 2960
waiting for: 3 seconds
Pulling request 2633, total API requests so far = 2961
waiting for: 3 seconds
Pulling request 2634, total API requests so far = 2962
waiting for: 1 seconds
Pulling request 2635, total API requests 

Pulling request 2728, total API requests so far = 3056
waiting for: 1 seconds
Pulling request 2729, total API requests so far = 3057
waiting for: 3 seconds
Pulling request 2730, total API requests so far = 3058
waiting for: 1 seconds
Pulling request 2731, total API requests so far = 3059
waiting for: 3 seconds
Pulling request 2732, total API requests so far = 3060
waiting for: 1 seconds
Pulling request 2733, total API requests so far = 3061
waiting for: 3 seconds
Pulling request 2734, total API requests so far = 3062
waiting for: 1 seconds
Pulling request 2735, total API requests so far = 3063
waiting for: 1 seconds
Pulling request 2736, total API requests so far = 3064
waiting for: 2 seconds
Pulling request 2737, total API requests so far = 3065
waiting for: 2 seconds
Pulling request 2738, total API requests so far = 3066
waiting for: 3 seconds
Pulling request 2739, total API requests so far = 3067
waiting for: 1 seconds
Pulling request 2740, total API requests so far = 3068
waiting f

Pulling request 2834, total API requests so far = 3162
waiting for: 3 seconds
Pulling request 2835, total API requests so far = 3163
waiting for: 3 seconds
Pulling request 2836, total API requests so far = 3164
waiting for: 1 seconds
Pulling request 2837, total API requests so far = 3165
waiting for: 2 seconds
Pulling request 2838, total API requests so far = 3166
waiting for: 2 seconds
Pulling request 2839, total API requests so far = 3167
waiting for: 3 seconds
Pulling request 2840, total API requests so far = 3168
waiting for: 1 seconds
Pulling request 2841, total API requests so far = 3169
waiting for: 2 seconds
Pulling request 2842, total API requests so far = 3170
waiting for: 2 seconds
Pulling request 2843, total API requests so far = 3171
waiting for: 1 seconds
Pulling request 2844, total API requests so far = 3172
waiting for: 1 seconds
Pulling request 2845, total API requests so far = 3173
waiting for: 2 seconds
Pulling request 2846, total API requests so far = 3174
waiting f

...

waiting for: 2 seconds
Pulling request 3893, total API requests so far = 4221
waiting for: 2 seconds
Pulling request 3894, total API requests so far = 4222
waiting for: 2 seconds
Pulling request 3895, total API requests so far = 4223
waiting for: 3 seconds
Pulling request 3896, total API requests so far = 4224
waiting for: 3 seconds
Pulling request 3897, total API requests so far = 4225
waiting for: 2 seconds
Pulling request 3898, total API requests so far = 4226
waiting for: 3 seconds
Pulling request 3899, total API requests so far = 4227
waiting for: 2 seconds
Pulling request 3900, total API requests so far = 4228
waiting for: 1 seconds
Pulling request 3901, total API requests so far = 4229
waiting for: 3 seconds
Pulling request 3902, total API requests so far = 4230
waiting for: 2 seconds
Pulling request 3903, total API requests so far = 4231
waiting for: 3 seconds
Pulling request 3904, total API requests so far = 4232
waiting for: 3 seconds
Pulling request 3905, total API requests 

Pulling request 3998, total API requests so far = 4326
waiting for: 1 seconds
Pulling request 3999, total API requests so far = 4327
waiting for: 3 seconds
Pulling request 4000, total API requests so far = 4328
waiting for: 3 seconds
Pulling request 4001, total API requests so far = 4329
waiting for: 2 seconds
Pulling request 4002, total API requests so far = 4330
waiting for: 3 seconds
Pulling request 4003, total API requests so far = 4331
waiting for: 2 seconds
Pulling request 4004, total API requests so far = 4332
waiting for: 3 seconds
Pulling request 4005, total API requests so far = 4333
waiting for: 2 seconds
Pulling request 4006, total API requests so far = 4334
waiting for: 1 seconds
Pulling request 4007, total API requests so far = 4335
waiting for: 3 seconds
Pulling request 4008, total API requests so far = 4336
waiting for: 3 seconds
Pulling request 4009, total API requests so far = 4337
waiting for: 1 seconds
Pulling request 4010, total API requests so far = 4338
waiting f

Pulling request 4104, total API requests so far = 4432
waiting for: 3 seconds
Pulling request 4105, total API requests so far = 4433
waiting for: 2 seconds
Pulling request 4106, total API requests so far = 4434
waiting for: 1 seconds
Pulling request 4107, total API requests so far = 4435
waiting for: 3 seconds
Pulling request 4108, total API requests so far = 4436
waiting for: 2 seconds
Pulling request 4109, total API requests so far = 4437
waiting for: 3 seconds
Pulling request 4110, total API requests so far = 4438
waiting for: 3 seconds
Pulling request 4111, total API requests so far = 4439
waiting for: 3 seconds
Pulling request 4112, total API requests so far = 4440
waiting for: 1 seconds
Pulling request 4113, total API requests so far = 4441
waiting for: 3 seconds
Pulling request 4114, total API requests so far = 4442
waiting for: 3 seconds
Pulling request 4115, total API requests so far = 4443
waiting for: 2 seconds
Pulling request 4116, total API requests so far = 4444
waiting f

waiting for: 1 seconds
Pulling request 4210, total API requests so far = 4538
waiting for: 3 seconds
Pulling request 4211, total API requests so far = 4539
waiting for: 2 seconds
Pulling request 4212, total API requests so far = 4540
waiting for: 2 seconds
Pulling request 4213, total API requests so far = 4541
waiting for: 1 seconds
Pulling request 4214, total API requests so far = 4542
waiting for: 1 seconds
Pulling request 4215, total API requests so far = 4543
waiting for: 3 seconds
Pulling request 4216, total API requests so far = 4544
waiting for: 3 seconds
Pulling request 4217, total API requests so far = 4545
waiting for: 3 seconds
Pulling request 4218, total API requests so far = 4546
waiting for: 1 seconds
Pulling request 4219, total API requests so far = 4547
waiting for: 3 seconds
Pulling request 4220, total API requests so far = 4548
waiting for: 1 seconds
Pulling request 4221, total API requests so far = 4549
waiting for: 1 seconds
Pulling request 4222, total API requests 

Pulling request 4315, total API requests so far = 4643
waiting for: 3 seconds
Pulling request 4316, total API requests so far = 4644
waiting for: 3 seconds
Pulling request 4317, total API requests so far = 4645
waiting for: 1 seconds
Pulling request 4318, total API requests so far = 4646
waiting for: 2 seconds
Pulling request 4319, total API requests so far = 4647
waiting for: 3 seconds
Pulling request 4320, total API requests so far = 4648
waiting for: 3 seconds
Pulling request 4321, total API requests so far = 4649
waiting for: 2 seconds
Pulling request 4322, total API requests so far = 4650
waiting for: 3 seconds
Pulling request 4323, total API requests so far = 4651
waiting for: 1 seconds
Pulling request 4324, total API requests so far = 4652
waiting for: 1 seconds
Pulling request 4325, total API requests so far = 4653
waiting for: 2 seconds
Pulling request 4326, total API requests so far = 4654
waiting for: 3 seconds
Pulling request 4327, total API requests so far = 4655
waiting f